# 0. Setup Paths

In [1]:
import os
# list of requirement: tensorflow, matplotlib, yaml, tensorflow-io, matplotlib==3.2

In [2]:
FIRST_TIME_RUNNING = False
NEW_MODEL = False
TRAIN = False
EVAL = False
CHECKPOINT = 3
TEST_SINGLE_IMAGE = True
TEST_MULTIPLE_IMAGES = False
TEST_VIDEO = True


ROOT_DIR = r"C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Script\Models\TFODCourse\Tensorflow"
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'


In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_resnet50_640' 
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
PRETRAINED_MODEL_NAME = PRETRAINED_MODEL_URL.split('/')[-1].split('.')[0]
print(PRETRAINED_MODEL_NAME)

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8


In [ ]:
# CUSTOM_MODEL_NAME = 'my_ssd_resnet50_640_test' 
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
# PRETRAINED_MODEL_NAME = PRETRAINED_MODEL_URL.split('/')[-1].split('.')[0]
# print(PRETRAINED_MODEL_NAME)

In [ ]:
# CUSTOM_MODEL_NAME = 'my_ssd_resnet50_1024' 
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'
# PRETRAINED_MODEL_NAME = PRETRAINED_MODEL_URL.split('/')[-1].split('.')[0]
# print(PRETRAINED_MODEL_NAME)


In [ ]:
# CUSTOM_MODEL_NAME = 'my_ssd_mobnet_320'
# PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
# PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

In [ ]:
# CUSTOM_MODEL_NAME = 'my_centernet_hourglass104' 
# PRETRAINED_MODEL_NAME = 'centernet_hg104_512x512_coco17_tpu-8'
# PRETRAINED_MURL = 'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz'

In [4]:
IMG_DIR = 'images'

paths = {
    'WORKSPACE_PATH': os.path.join(ROOT_DIR, 'workspace'),
    'SCRIPTS_PATH': os.path.join(ROOT_DIR,'scripts'),
    'APIMODEL_PATH': os.path.join(ROOT_DIR,'models'),
    'ANNOTATION_PATH': os.path.join(ROOT_DIR, 'workspace','annotations'),
    'IMAGE_PATH': os.path.join(ROOT_DIR, 'workspace', IMG_DIR),
    'MODEL_PATH': os.path.join(ROOT_DIR, 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join(ROOT_DIR, 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join(ROOT_DIR, 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join(ROOT_DIR, 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join(ROOT_DIR, 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(ROOT_DIR, 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join(ROOT_DIR,'protoc'),
    'SPECIES_LIST_PATH': os.path.join(ROOT_DIR, 'workspace', IMG_DIR,'collectedimages'),
    'TRAIN_PATH' : os.path.join(ROOT_DIR, 'workspace', IMG_DIR, 'train'),
    'TEST_PATH' : os.path.join(ROOT_DIR, 'workspace', IMG_DIR, 'test'),
 }

In [5]:
files = {
    'PIPELINE_CONFIG':os.path.join(ROOT_DIR, 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [6]:
for path in paths.values():
        if not os.path.exists(path):
            if os.name == 'posix':
                !mkdir -p {path}
            if os.name == 'nt':
                !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [ ]:
# https://www.tensorflow.org/install/source_windows

In [7]:
def download_api_model():
    if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
        !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [8]:
def download_protoc():
    # Install Tensorflow Object Detection 
    if os.name=='posix':  
        !apt-get install protobuf-compiler
        !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
        
    if os.name=='nt':
        url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
        wget.download(url)
        !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
        !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
        os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
        !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
        !cd Tensorflow/models/research/slim && pip install -e . 

In [9]:
def verify():
    VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
    # Verify Installation
    !python {VERIFICATION_SCRIPT}

In [10]:
def download_pretrained_model():
    if os.name =='posix':
        !wget {PRETRAINED_MODEL_URL}
        !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
        !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
    if os.name == 'nt':
        if not os.path.exists(PRETRAINED_MODEL_NAME+'.tar.gz'):
            wget.download(PRETRAINED_MODEL_URL)
        !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
        !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

In [11]:
if FIRST_TIME_RUNNING == True:
    if os.name=='nt':
        !pip install wget
        import wget
    download_api_model()
    download_protoc()
    verify()
    download_pretrained_model()
elif os.name=='nt':
    import wget

if NEW_MODEL == True:
    download_pretrained_model()
    
import object_detection

# 2. Create Label Map

In [13]:
# species_list_path = r"C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Script\BBOX\BackUp"
species_list = os.listdir(paths['SPECIES_LIST_PATH'])
species_list_with_gender = []
for species in species_list:
    species_list_with_gender.append(species + '_MALE')
    species_list_with_gender.append(species + '_FEMALE')
print(species_list_with_gender)
labels = [{'name':species, 'id':i+1} for i, species in enumerate(species_list_with_gender)]
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

['Corinnomma-severum_MALE', 'Corinnomma-severum_FEMALE', 'Cosmophasis-lami_MALE', 'Cosmophasis-lami_FEMALE', 'Crossopriza-lyoni_MALE', 'Crossopriza-lyoni_FEMALE', 'Cyclosa-mulmeinensis_MALE', 'Cyclosa-mulmeinensis_FEMALE', 'Cyrtarachne-bufo_MALE', 'Cyrtarachne-bufo_FEMALE', 'Cyrtarachne-inaequalis_MALE', 'Cyrtarachne-inaequalis_FEMALE', 'Cyrtophora-exanthematica_MALE', 'Cyrtophora-exanthematica_FEMALE', 'Cyrtophora-moluccensis_MALE', 'Cyrtophora-moluccensis_FEMALE', 'Dolomedes-mizhoanus_MALE', 'Dolomedes-mizhoanus_FEMALE', 'Dolomedes-raptor_MALE', 'Dolomedes-raptor_FEMALE', 'Dolomedes-sulfureus_MALE', 'Dolomedes-sulfureus_FEMALE', 'Ebrechtella-tricuspidata_MALE', 'Ebrechtella-tricuspidata_FEMALE', 'Epeus-glorius_MALE', 'Epeus-glorius_FEMALE', 'Eriovixia-laglaizei_MALE', 'Eriovixia-laglaizei_FEMALE', 'Gasteracantha-kuhli_MALE', 'Gasteracantha-kuhli_FEMALE', 'Gasteracantha-sauteri_MALE', 'Gasteracantha-sauteri_FEMALE', 'Hamadruas-hieroglyphica_MALE', 'Hamadruas-hieroglyphica_FEMALE', 'Ha

# 3. Create TF records

In [14]:
def move_images(TESTING = True):
    from object_detection.utils import move_images
    move_images.move_images(paths['SPECIES_LIST_PATH'], paths['TRAIN_PATH'], paths['TEST_PATH'], testing=TESTING)

# move_images(TESTING = False)       

In [15]:
# # OPTIONAL IF RUNNING ON COLAB
# ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
# if os.path.exists(ARCHIVE_FILES):
#   !tar -zxvf {ARCHIVE_FILES}

In [16]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [17]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Traceback (most recent call last):
  File "C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Script\Models\TFODCourse\Tensorflow\scripts\generate_tfrecord.py", line 168, in <module>
    tf.app.run()
  File "c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\tensorflow\python\platform\app.py", line 36, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\absl\app.py", line 308, in run
    _run_main(main, args)
  File "c:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\absl\app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Script\Models\TFODCourse\Tensorflow\scripts\generate_tfrecord.py", line 158, in main
    tf_example = create_tf_example(group, path)
  File "C:\Code\ChungyuanProjects\SpiderDataset01\inaturalist\Script\Models\TFODCourse\Tensorflow\scripts\generate_tfrecord.py", line 132, in create_tf_example
    classes.

# 4. Copy Model Config to Training Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
# config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

In [ ]:
if TRAIN == True:
    !{command}

# 7. Evaluate the Model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

In [ ]:
if EVAL == True:
    !{command}

# 8. Load Train Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], f'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8.5 
(myenv) PS ./Tensorflow\workspace\models\my_ssd_resnet50_640\eval> tensorboard --logdir=.

go to 'http://localhost:6006/'

# 9. Detect from an Image

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
if TEST_SINGLE_IMAGE == True:
    IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'Chikunia-nigra_18.jpg')
    img = cv2.imread(IMAGE_PATH)
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    #draw rectangle using number arrays from detections['detection_boxes']
    plt.imshow(image_np_with_detections)
    plt.show()

    # # image_np_with_detections = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
    # plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    # plt.show()

In [ ]:
TEST_IMAGES_PATH = os.path.join(paths['IMAGE_PATH'], 'test')

In [ ]:
if TEST_MULTIPLE_IMAGES == True:
    for test_image in os.listdir(TEST_IMAGES_PATH):
        if test_image.endswith('.xml'):
            continue
        IMAGE_PATH = os.path.join(TEST_IMAGES_PATH, test_image)
        img = cv2.imread(IMAGE_PATH)
        image_np = np.array(img)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                    for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=.8,
                    agnostic_mode=False)
        #Convert color from BGR to RGB then cv2.imshow
        image_np_with_detections = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
        cv2.imshow('image', image_np_with_detections)
        # plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
        # plt.show()

        #wait key, if press Y then continue, N then stop loop
        key = cv2.waitKey(0)
        if key == ord('y'):
            continue
        else:
            #destroy window
            cv2.destroyAllWindows()
            break

In [ ]:
'''
INFO:tensorflow:Performing evaluation on 90 images.
I0804 22:46:36.371927 20172 coco_evaluation.py:293] Performing evaluation on 90 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0804 22:46:36.373924 20172 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
I0804 22:46:36.379919 20172 coco_tools.py:138] DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.14s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.096
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.017
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.234
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.474
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.474
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.474
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
INFO:tensorflow:Eval metrics at step 2000
I0804 22:46:36.799071 20172 model_lib_v2.py:1015] Eval metrics at step 2000
INFO:tensorflow:        + DetectionBoxes_Precision/mAP: 0.034056
I0804 22:46:36.808483 20172 model_lib_v2.py:1018]       + DetectionBoxes_Precision/mAP: 0.034056
INFO:tensorflow:        + DetectionBoxes_Precision/mAP@.50IOU: 0.096185
I0804 22:46:36.810483 20172 model_lib_v2.py:1018]       + DetectionBoxes_Precision/mAP@.50IOU: 0.096185
INFO:tensorflow:        + DetectionBoxes_Precision/mAP@.75IOU: 0.016757
I0804 22:46:36.812483 20172 model_lib_v2.py:1018]       + DetectionBoxes_Precision/mAP@.75IOU: 0.016757
INFO:tensorflow:        + DetectionBoxes_Precision/mAP (small): -1.000000
I0804 22:46:36.813498 20172 model_lib_v2.py:1018]       + DetectionBoxes_Precision/mAP (small): -1.000000
INFO:tensorflow:        + DetectionBoxes_Precision/mAP (medium): 0.034063
I0804 22:46:36.815499 20172 model_lib_v2.py:1018]       + DetectionBoxes_Precision/mAP (medium): 0.034063
INFO:tensorflow:        + DetectionBoxes_Precision/mAP (large): -1.000000
I0804 22:46:36.816741 20172 model_lib_v2.py:1018]       + DetectionBoxes_Precision/mAP (large): -1.000000
INFO:tensorflow:        + DetectionBoxes_Recall/AR@1: 0.234444
I0804 22:46:36.819666 20172 model_lib_v2.py:1018]       + DetectionBoxes_Recall/AR@1: 0.234444
INFO:tensorflow:        + DetectionBoxes_Recall/AR@10: 0.474444
I0804 22:46:36.821847 20172 model_lib_v2.py:1018]       + DetectionBoxes_Recall/AR@10: 0.474444
INFO:tensorflow:        + DetectionBoxes_Recall/AR@100: 0.474444
I0804 22:46:36.822846 20172 model_lib_v2.py:1018]       + DetectionBoxes_Recall/AR@100: 0.474444
INFO:tensorflow:        + DetectionBoxes_Recall/AR@100 (small): -1.000000
I0804 22:46:36.824847 20172 model_lib_v2.py:1018]       + DetectionBoxes_Recall/AR@100 (small): -1.000000
INFO:tensorflow:        + DetectionBoxes_Recall/AR@100 (medium): 0.474444
I0804 22:46:36.825846 20172 model_lib_v2.py:1018]       + DetectionBoxes_Recall/AR@100 (medium): 0.474444
INFO:tensorflow:        + DetectionBoxes_Recall/AR@100 (large): -1.000000
I0804 22:46:36.828293 20172 model_lib_v2.py:1018]       + DetectionBoxes_Recall/AR@100 (large): -1.000000
INFO:tensorflow:        + Loss/localization_loss: 0.258691
I0804 22:46:36.829552 20172 model_lib_v2.py:1018]       + Loss/localization_loss: 0.258691
INFO:tensorflow:        + Loss/classification_loss: 1.554604
I0804 22:46:36.830848 20172 model_lib_v2.py:1018]       + Loss/classification_loss: 1.554604
INFO:tensorflow:        + Loss/regularization_loss: 0.146907
I0804 22:46:36.832127 20172 model_lib_v2.py:1018]       + Loss/regularization_loss: 0.146907
INFO:tensorflow:        + Loss/total_loss: 1.960202
I0804 22:46:36.835216 20172 model_lib_v2.py:1018]       + Loss/total_loss: 1.960202
'''

# 10. Real Time Detections from your Webcam

In [ ]:
# !pip uninstall opencv-python-headless -y

In [ ]:
STREAM_URL = "https://192.168.68.109:8080/video"

cap = cv2.VideoCapture(STREAM_URL)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

if TEST_VIDEO == True:
    while cap.isOpened(): 
        ret, frame = cap.read()
        image_np = np.array(frame)
        
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)
        
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                    for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=.8,
                    agnostic_mode=False)

        cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

# 10. Freezing the Graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['OUTPUT_PATH'])

In [ ]:
print(command)

In [ ]:
!{command}

# 11. Conversion to TFJS

In [ ]:
# !pip install tensorflowjs

In [ ]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(paths['OUTPUT_PATH'], 'saved_model'), paths['TFJS_PATH'])

In [ ]:
print(command)

In [ ]:
# !{command}

In [ ]:
# Test Code: https://github.com/nicknochnack/RealTimeSignLanguageDetectionwithTFJS

# 12. Conversion to TFLite

In [ ]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [ ]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])

In [ ]:
print(command)

In [ ]:
# !{command}

In [ ]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', 'detect.tflite')

In [ ]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )

In [ ]:
print(command)

In [ ]:
# !{command}

# 13. Zip and Export Models 

In [ ]:
# !tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')